## libs 

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

#### 1. Đặc trưng Cấp User (User-level Features)

In [4]:
# Giả sử dữ liệu đã chuẩn hóa: dùng train_processed.csv đã có session_id, datetime...

train = pd.read_csv('train_processed.csv')
# Nếu chưa có cột datetime: train['datetime'] = pd.to_datetime(train['timestamp'], unit='ms')

# Tổng số event từng loại theo user
user_stats = train.groupby('visitorid')['event'].value_counts().unstack(fill_value=0)
user_stats.columns = ['num_addtocart', 'num_transaction', 'num_view']

# Tỷ lệ chuyển đổi
user_stats['view2cart_rate'] = user_stats['num_addtocart'] / user_stats['num_view'].replace(0,1)
user_stats['cart2tran_rate'] = user_stats['num_transaction'] / user_stats['num_addtocart'].replace(0,1)
user_stats['view2tran_rate'] = user_stats['num_transaction'] / user_stats['num_view'].replace(0,1)

# Số lần view trung bình trước khi mua
def view_before_buy(df):
    if df['num_transaction']>0:
        return df['num_view']/df['num_transaction']
    return df['num_view']
user_stats['avg_view_before_buy'] = user_stats.apply(view_before_buy, axis=1)

# Số session, độ tần suất
num_session = train.groupby('visitorid')['session_id'].nunique()
user_stats['num_session'] = num_session

# Độ mới (recency), độ thường xuyên (frequency)
last_event_time = train.groupby('visitorid')['datetime'].max()
first_event_time = train.groupby('visitorid')['datetime'].min()
user_stats['recency_days'] = (pd.to_datetime('now') - pd.to_datetime(last_event_time)).dt.days
user_stats['active_days'] = (pd.to_datetime(last_event_time) - pd.to_datetime(first_event_time)).dt.days + 1
user_stats['session_per_day'] = user_stats['num_session']/user_stats['active_days'].replace(0,1)

# Sự đa dạng lượt xem
item_diversity = train.groupby('visitorid')['itemid'].nunique()
user_stats['item_diversity'] = item_diversity

user_stats = user_stats.reset_index()
user_stats.to_csv('user_behavior_features.csv', index=False)


#### 2. Đặc trưng Cấp Item (Item-level Features)

In [5]:
# Tổng view, addtocart, transaction cho mỗi sản phẩm
item_stats = train.groupby('itemid')['event'].value_counts().unstack(fill_value=0)
item_stats.columns = ['num_addtocart', 'num_transaction', 'num_view']

item_stats['view2cart_rate'] = item_stats['num_addtocart'] / item_stats['num_view'].replace(0,1)
item_stats['cart2tran_rate'] = item_stats['num_transaction'] / item_stats['num_addtocart'].replace(0,1)
item_stats['view2tran_rate'] = item_stats['num_transaction'] / item_stats['num_view'].replace(0,1)

item_stats['user_diversity'] = train.groupby('itemid')['visitorid'].nunique()

item_stats = item_stats.reset_index()
item_stats.to_csv('item_behavior_features.csv', index=False)


#### 3. Đặc trưng bổ sung (cao cấp, cho user hoặc session):

In [ ]:
def mean_time_view2buy(gdf):
    # Chỉ tính trên user có transaction
    times = []
    for itemid in gdf['itemid'].unique():
        item_df = gdf[gdf['itemid']==itemid]
        views = item_df[item_df['event']=='view']['datetime']
        buys = item_df[item_df['event']=='transaction']['datetime']
        if len(buys)>0 and len(views)>0:
            tmin = abs(pd.to_datetime(buys.values[0]) - pd.to_datetime(views.values[0])).total_seconds()/60
            times.append(tmin)
    return np.mean(times) if times else np.nan

user_stats['mean_min_view2buy'] = (
    train.groupby('visitorid')
    .apply(mean_time_view2buy)
)


#### 4. Lưu output đặc trưng

In [ ]:
# # Lưu user features
# user_stats.to_csv('user_behavior_features.csv', index=False)

# # Lưu item features
# item_stats.to_csv('item_behavior_features.csv', index=False)